In [3]:
import praw 
import pandas as pd
import datetime as dt
import numpy as np

In [4]:
reddit=praw.Reddit(client_id='_Ij9InJJ1MqC-Q',client_secret='9XF46Dd9icSEmAghFivwt9J9uGU',user_agent='Reddit WebScraping',username='anushkanarula01',password='')

In [5]:
subreddit = reddit.subreddit('india')

In [6]:
flair_dict={0:'Political',1:'Non-political',2:'Reddiquette',3:'AskIndia',4:'Science & Technology',5:'Policy & Economy',6:'Finance & Business',7:'Sports and food',8:'Photography',9:'AMA'}

In [9]:
topics_dict = { "title":[],"score":[],"id":[],"url":[],"comms_num": [],"created": [],"body":[],"flair":[]}
for flair in flair_dict:
    for submission in subreddit.search(flair_dict[flair],limit=1000):
        topics_dict["title"].append(submission.title)
        topics_dict["score"].append(submission.score)
        topics_dict["id"].append(submission.id)
        topics_dict["url"].append(submission.url)
        topics_dict["comms_num"].append(submission.num_comments)
        topics_dict["created"].append(submission.created)
        topics_dict["body"].append(submission.selftext)
        topics_dict["flair"].append(flair)

topics_data = pd.DataFrame(topics_dict)
topics_data.to_csv('data.csv',index=False)
len(topics_data['title'])

1950

In [10]:
df=pd.read_csv('data.csv')

In [11]:
df.head()

,title,score,id,url,comms_num,created,body,flair
0,Pitting a community against a political party ...,193,futac9,https://www.reddit.com/r/india/comments/futac9...,73,1.586034e+09,First of all let me start by saying it was stu...,0
1,Hit by backlash over posts on lack of medical ...,406,fpaj1w,https://theprint.in/india/hit-by-backlash-over...,67,1.585254e+09,NaN,0
2,A new political party gave a full front page a...,733,ff8sth,https://i.redd.it/yjo9wpy38el41.jpg,146,1.583678e+09,NaN,0
3,"In this trying time, let's try to be a little ...",216,flm79b,https://www.reddit.com/r/india/comments/flm79b...,56,1.584697e+09,I had a shocking interaction this morning. A b...,0
4,Indian States / UTs if renamed for countries w...,182,flwd7r,https://i.redd.it/hk9o11b8dun41.png,20,1.584745e+09,NaN,0
